<a href="https://colab.research.google.com/github/Yadav122/Medical-report-extractor/blob/main/laboratory_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
!pip install pydantic

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
import os
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = api_key

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [ ]:
llm.invoke("hi how ae you")

AIMessage(content="I'm doing well, thanks for asking! \n\nI'm an AI language model, so I don't have feelings, but I'm here to assist you with any questions or tasks you may have. \n\nHow can I help you today? \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-268da679-899d-473e-b8ff-b874f5d89854-0')

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
from langchain_google_genai import HarmBlockThreshold, HarmCategory

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",safety_settings=safety_settings,temperature=0)

In [ ]:
image_message = {
    "type": "image_url",
    "image_url": {"url": "https://images.drlogy.com/assets/uploads/lab/image/cbc-test-report-format-example-sample-template-drlogy-lab-report.webp"},
}
text_message = {
    "type": "text",
    "text": "extract the schema for this cbc repor and fille the schema whith appropriate values in json forat",
}

In [ ]:
message = HumanMessage(content=[text_message, image_message])

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field

class Format(BaseModel):
  value: Optional[int |float ] = Field(description="value of the particular attribute fromt the report")
  unit: Optional[str] = Field(description="unit of the value extracted if no unit "" ")
  range: Optional[str] = Field(description="range of the value the report specifies ")

class CBC(BaseModel):
    WBC_count: Format = Field( description="White Blood Cell count ")
    RBC_count: Optional[Format] = Field( description="Red Blood Cell count ")
    Hemoglobin: Format = Field(description="Hemoglobin concentration from the report")
    Hematocrit: Format = Field(description="Hematocrit percentage")
    MCV: Format = Field(description="Mean Corpuscular Volume ")
    MCH: Format = Field(description="Mean Corpuscular Hemoglobin ")
    MCHC: Format = Field( description="Mean Corpuscular Hemoglobin Concentration ")
    RDW: Format = Field(description="Red Cell Distribution Width (size and shape)")
    Platelet_count: Format = Field(description="Platelet count")
    MPV: Optional[Format] = Field(description="Mean Platelet Volume ")

In [ ]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=CBC)

In [ ]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Format": {"properties": {"value": {"anyOf": [{"type": "integer"}, {"type": "number"}, {"type": "null"}], "description": "value of the particular attribute fromt the report", "title": "Value"}, "unit": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "unit of the value extracted if no unit  ", "title": "Unit"}, "range": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "range of the value the report specifies ", "title": "Range"}}, "required": ["value", "unit", "range"], "title": "Format", "typ

In [ ]:
template="you are a json data exctractor i need you to extract data from the images of medical reports in the given schema.\nschema:{format_instructions}\n\n"

In [ ]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate(template="you are a json data exctractor i need you to extract data from the images of medical reports in the given schema.\nschema:{format_instructions}\n\n",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
text = template.format()

In [ ]:
text

'you are a json data exctractor i need you to extract data from the images of medical reports in the given schema.\nschema:The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Format": {"properties": {"value": {"anyOf": [{"type": "integer"}, {"type": "number"}, {"type": "null"}], "description": "value of the particular attribute fromt the report", "title": "Value"}, "unit": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "unit of the value extracted if no unit  ", "title": "Unit"}, "range": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "rang

In [ ]:
#Processing Images
from langchain_core.messages import SystemMessage
def pass_img(url):
  image_message = {
    "type": "image_url",
    "image_url": {"url": url},
  }
  text_message = {
      "type": "text",
      "text": """you are a medical lab report assistant your job is to extract data from lab reorts and store them in proper json format""",
  }
  format_inst = {
    "type": "text",
    "text": text,
  }
  sysmsg = SystemMessage(content=[text_message])
  message = HumanMessage(content=[image_message, format_inst])
  return [sysmsg,message]

In [ ]:
from langchain_core.messages import SystemMessage
def pass_img(url):
  image_message = {
    "type": "image_url",
    "image_url": {"url": url},
  }
  text_message = {
      "type": "text",
      "text": """you are a medical lab report assistant your job is to extract data from lab reorts and store them in proper json format""",
  }
  format_inst = {
    "type": "text",
    "text": text,
  }
  sysmsg = SystemMessage(content=[text_message])
  message = HumanMessage(content=[image_message, format_inst])
  return [sysmsg,message]

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda
# Combining Everything in a Runnable Chain  and used to create executable workflows.

In [ ]:
pass_img

<function __main__.pass_img(url)>

In [ ]:
#Combines the image processing function, language model, and output parser into a single executable chain using the pipe (|) operator
chain = RunnableLambda(pass_img) | llm | RunnableLambda(lambda x: x.content) | parser

In [ ]:
RunnableLambda(pass_img)

RunnableLambda(pass_img)

In [ ]:
chain = RunnableLambda(pass_img)

In [ ]:
result = chain.invoke("https://images.drlogy.com/assets/uploads/lab/image/cbc-test-report-format-example-sample-template-drlogy-lab-report.webp")

In [ ]:
#Prints the final result, which should be the extracted data in JSON format validated against the CBC schema
result

[SystemMessage(content=[{'type': 'text', 'text': 'you are a medical lab report assistant your job is to extract data from lab reorts and store them in proper json format'}]),
 HumanMessage(content=[{'type': 'image_url', 'image_url': {'url': 'https://images.drlogy.com/assets/uploads/lab/image/cbc-test-report-format-example-sample-template-drlogy-lab-report.webp'}}, {'type': 'text', 'text': 'you are a json data exctractor i need you to extract data from the images of medical reports in the given schema.\nschema:The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Format": {"prop